In [1]:
from langchain.agents import ZeroShotAgent, AgentExecutor
from langchain.chains import LLMChain
from langchain.agents import load_tools, Tool
from langchain.chat_models import ChatOpenAI

from typing import List

import os
from dotenv import load_dotenv
load_dotenv()

In [2]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo",
                 temperature=0.7, 
                 openai_api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
def _create_agent(tools: List[Tool], prefix: str, suffix: str):
    """Helper function to create an agent with specified tools and prompt"""
    prompt = ZeroShotAgent.create_prompt(
        tools,
        prefix=prefix,
        suffix=suffix,
        input_variables=["input", "agent_scratchpad"]
    )
    
    llm_chain = LLMChain(llm=llm, prompt=prompt)
    
    agent = ZeroShotAgent(
        llm_chain=llm_chain,
        tools=tools,
        verbose=True
    )
    
    return AgentExecutor.from_agent_and_tools(
        agent=agent,
        tools=tools,
        verbose=True
    )

In [8]:
def create_math_agent():
    tools = load_tools(['llm-math'], llm=llm)

    prefix = """You are responsible for calculating the exams scores for a class of students.

    You will receive a list of student grades between 0 and 10 delimitted by ## and separated by ";". 
    
    You have access to the following tools:"""
    
    suffix = """Return the grades average: {input}
    {agent_scratchpad}"""
    
    return _create_agent(tools, prefix, suffix)


agent_exc = create_math_agent()

agent_exc.run("""##9;2;5;8;9;10;3;6##""")



> Entering new AgentExecutor chain...
Question: What is the average of the student grades?
Thought: I need to add up all the grades and divide by the total number of grades.
Action: Calculator
Action Input: (9 + 2 + 5 + 8 + 9 + 10 + 3 + 6) / 8
Observation: Answer: 6.5
Thought:I now know the final answer
Final Answer: 6.5

> Finished chain.


'6.5'

In [11]:
# from datetime import datetime


# def validate_date(date_string: str, date_format: str = "%Y-%m-%d"):
#     try:
#         datetime.strptime(date_string, date_format)
#         return True
#     except ValueError:
#         return False


def create_wikipedia_news_agent():
    tools = load_tools(['wikipedia'], llm=llm)

    prefix = """You are responsible for summarizing the news in the wikipedia relative to a specific date.

    You will receive a date. 

    The date will be provided in the format ##YYYY-MM-DD##.

    If the date is not provided, the agent will use the current date.

    Validate the date format. If the date is invalid, the agent will return an error message: "Invalid date format".

    Return only a list of the news found in bullet points, nothing else.

    You have access to the following tools:"""
    
    suffix = """Return in bullet points the news found: {input}
    {agent_scratchpad}"""
    
    return _create_agent(tools, prefix, suffix)


agent_exc = create_wikipedia_news_agent()

agent_exc.run("##05/01/2022##")



> Entering new AgentExecutor chain...
Question: What were the news on 05/01/2022?
Thought: I should use Wikipedia to search for news on that specific date.
Action: Wikipedia
Action Input: May 1, 2022
Observation: Page: May 1
Summary: May 1 is the 121st day of the year (122nd in leap years) in the Gregorian calendar;  244 days remain until the end of the year.



Page: 2022
Summary: 2022 (MMXXII) was a common year starting on Saturday of the Gregorian calendar, the 2022nd year of the Common Era (CE) and Anno Domini (AD) designations, the 22nd  year of the 3rd millennium and the 21st century, and the  3rd   year of the 2020s decade.  
The year began with another wave in the COVID-19 pandemic, with Omicron spreading rapidly and becoming the dominant variant of the SARS-CoV-2 virus worldwide. Tracking a decrease in cases and deaths, 2022 saw the removal of most COVID-19 restrictions and the reopening of international borders in the vast majority of countries, while the global rollout of 

'- 2022 started with the Omicron variant becoming dominant in the COVID-19 pandemic\n- The global economic recovery continued, with many countries experiencing inflation surges\n- Natural disasters, including hurricanes and a powerful volcano eruption, occurred\n- Wars and armed conflicts, such as the Russian invasion of Ukraine, dominated the year\n- The May 2022 Canadian derecho caused widespread damage and was the sixth-costliest event in Canadian history'